# 中華民國國旗
### 紅地 = red, 左上角青天 = blue_green, 白日 = white
#### 旗面尺寸: red_w = 900, red_h= w * 2/3
* 旗面之橫度與縱度為三與二之比
#### 青天尺寸: bg_w = red_w / 2, bg_h = red_h / 2
* 青天為長方形，其面積為全旗之四分之一
#### 白日尺寸: wh_r = (red_w / 2) / 8, center_cw =bg_w/2, center_ch=bg_h/2
* 白日體圓心位於長方形青天縱橫平分線之交點上
* 白日體半徑與青色長方形之橫長為一與八之比
#### 青底圓形尺寸: bg_r = wh_r * 2, bg_rw = (wh_r * 2) / 15
* 青底圓形之圓心為白日體之圓心
* 白日體半徑與青底圓形半徑為一與三之比
#### 12到白尖角光芒: wh_tr = wh_r * 3, angle = 30, firs_point = ((center_cw + wh_tr), center_ch)
* 白日體圓心至白尖角光芒頂，其長度與白日體半徑，為二與一之比
* 白日與十二道白尖角光芒間之青圈，其寬度等於白日體直徑十五分之一
* 每道白尖角光芒之頂角為三十度，十二角為三百六十度
* 白尖角光芒之上下左右排列應正對北南西東方向，其餘均勻排列


In [1]:
import cv2
import numpy as np


In [6]:
red_w = 900
red_h = int((red_w *2) / 3)
red_src = np.ones((red_h, red_w, 3), np.uint8) * 255

cv2.imshow('img', red_src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# 轉換成CMYK
bgr_norm = red_src.astype(float) / 255.0
# 計算 K 通道
K = 1 - np.max(bgr_norm, axis=2)

# 防止除以0,設定一個最小閾值
denominator = 1 - K
denominator[denominator == 0] = 1e-10

# 計算 C、M、Y 通道
C = (1 - bgr_norm[..., 2] - K) / denominator
M = (1 - bgr_norm[..., 1] - K) / denominator
Y = (1 - bgr_norm[..., 0] - K) / denominator



In [ ]:
# 紅底
red_C = np.zeros_like(C) * 0.0
red_M = np.ones_like(M) * 1.0
red_Y = np.ones_like(Y) * 1.0
red_K = np.ones_like(K) * 0.1

red_b = (1 - red_Y) * (1 - red_K) * 255
red_g = (1 - red_M) * (1 - red_K) * 255
red_r = (1 - red_C) * (1 - red_K) * 255
red_bgr = np.stack([red_b, red_g, red_r], axis=-1).astype(np.uint8)


cv2.imshow('red', red_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 青天
bg_C = np.ones_like(C) * 1.0
bg_M = np.ones_like(M) * 0.8
bg_Y = np.zeros_like(Y) * 0.0
bg_K = np.ones_like(K) * 0.2

bg_b = (1 - bg_Y) * (1 - bg_K) * 255
bg_g = (1 - bg_M) * (1 - bg_K) * 255
bg_r = (1 - bg_C) * (1 - bg_K) * 255
bg_bgr = np.stack([bg_b, bg_g, bg_r], axis=-1).astype(np.uint8)


cv2.imshow('blud_greed', bg_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = np.zeros((700, 700, 3), dtype=np.uint8)
center = (350, 350)
radius = 300
angle_half = 15
num_angles = 12
angle_step = 360 / num_angles
inner_length = 100  # 尖角內部線段長度，可調整

def get_point(center, length, angle_deg):
    angle_rad = np.deg2rad(angle_deg)
    x = int(center[0] + length * np.cos(angle_rad))
    y = int(center[1] - length * np.sin(angle_rad))
    return (x, y)

for i in range(num_angles):
    vertex_angle = i * angle_step
    vertex = get_point(center, radius, vertex_angle)  # 300距離圓心點(第一頂點)

    # 兩條邊角度：頂點角度加減15度，方向往中心方向 (反方向長度100)
    side_angle1 = vertex_angle + 180 - angle_half
    side_angle2 = vertex_angle + 180 + angle_half

    side_point1 = get_point(vertex, inner_length, side_angle1)
    side_point2 = get_point(vertex, inner_length, side_angle2)

    # 三角形頂點座標（第一頂點 + 往內兩邊點）
    triangle = np.array([vertex, side_point1, side_point2], dtype=np.int32).reshape((-1,1,2))

    cv2.fillPoly(img, [triangle], (255, 255, 255))

cv2.circle(img, center, 5, (0, 0, 255), -1)
cv2.imshow("Outer Vertex 300", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
